**Name:** Disaster Tweet Classifier

**Author:** Sharome Burton

**Date:** 07/19/2021

**Description:** Machine learning model used to predict whether a tweet is about a real disaster or not.

## 1. Problem definition
> How well can we predict whether a tweet is about a disaster or not?

## 2. Data
   * `train.csv` - the training set
   * `test.csv` - the test set
    
 source: https://www.kaggle.com/c/nlp-getting-started/data

   
## 3. Evaluation 

> **Goal:** Predict the whether a tweet is about a disaster or not with >80% accuracy.

## 4. Features

   * id - a unique identifier for each tweet
   * text - the text of the tweet
   * location - the location the tweet was sent from (may be blank)
   * keyword - a particular keyword from the tweet (may be blank)
   * target - in train.csv only, this denotes whether a tweet is about a real disaster (1) or not (0)


In [ ]:
### Import libraries

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

## Packages for basic text processing
import re
import string 

## Visualization tools
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

## Import Logistic Regression function for model training from linear model module of sklearn package
from sklearn.linear_model import LogisticRegression

## Import functions for evaluation from  model selection module of sklearn package
from sklearn.metrics import confusion_matrix, accuracy_score

## Import functions for model selection from model selection module of sklearn package
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

## Import functions for text vectorization from feature extraction module of sklearn package
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

## Import function from NLTK package for Text tokenization and Normalization
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

## Ignore warnings
import warnings
warnings.filterwarnings('ignore')

## Loading data

In [ ]:
tweets_raw = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
tweets_raw_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

## Exploratory Data Analysis (EDA)



In [ ]:
tweets_raw

In [ ]:
# Tweets about real disaster
tweets_raw[tweets_raw["target"] == 1]["text"].values[:5]


In [ ]:
# Tweets not about real disaster
tweets_raw[tweets_raw["target"] == 0]["text"].values[:5]


In [ ]:
# Check for missing data
tweets_raw.isna().sum()

In [ ]:
# Discarding columns except text and target
tweets=tweets_raw[['text','target']]

In [ ]:
# Number of occurences of real disasters
sns.countplot(x=tweets["target"])

In [ ]:
print(f'{tweets.target[tweets.target==1].count()/tweets.target.count()*100:.2f} % of tweets are labeled as disaster tweets in data')

In [ ]:
# Check for missing data
tweets.isna().sum()

## Splitting dataset into training and validation sets

In [ ]:
# Splitting data set into 80:20 ratio
train, test = train_test_split(tweets,test_size=0.25,random_state=8)

## Text Cleaning and Normalization

We will:

 *  Lowercase all words in text
 *  Remove newline characters if any in text
 *  Remove punctuations
 *  Remove url's and links from text
 *   Remove tags from text
 *  Remove multiple spaces from text
 *   Remove special characters
 *  Remove stop words from text
 *   Apply stemming to normalize the text




In [ ]:
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
def lower_text(text):
    """
        function to convert text into lowercase
        input: text
        output: cleaned text
    """
    text = text.lower() # lowering
    return text
    
def remove_newline(text):
    """
        function to remove new line characters in text
        input: text
        output: cleaned text
    """
    text = re.sub(r'\n',' ', text)
    return text

def remove_punctuations(text):
    """
        function to remove punctuations from text
        input: text
        output: cleaned text
    """    
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return text


def remove_links(text):
    """
        function to links and urls from text
        input: text
        output: cleaned text
    """    
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', text, flags=re.MULTILINE)
    
    return text

def remove_tags(text):
    """
        function to remove references and hashtags from text
        input: text
        output: cleaned text
    """    
    text = re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|(\w+:\/\/\S+)","",text)
    return text
    
def remove_multiplespaces(text):
    """
        function to remove multiple spaces from text
        input: text
        output: cleaned text
    """    
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    return text

def remove_specialchars(text):
    """
        function to remove special characters from text
        input: text
        output: cleaned text
    """    
    text = re.sub(r'\W', ' ', text)
    return text

def remove_stopwords(text):
    """
        function to tokenize the words using nltk word tokenizer and remove the stop words using nltk package's english stop words
        input: text
        output: cleaned text
    """    
    text = ' '.join([word for word in word_tokenize(text) if word not in stop_words])
    return text

def word_stemming(text):
    """
        function to perform stemming using porter stemmer from nltk package
        input: text
        output: cleaned text
    """        
    text=' '.join([stemmer.stem(word) for word in word_tokenize(text)])
    return text

### Cleaning and normalizing test dataset

In [ ]:
# Covert text to lowercase
train.text=train.text.apply(lambda text: lower_text(text))

# Remove newlines
train.text=train.text.apply(lambda text: remove_newline(text))

# Remove punctuations
train.text=train.text.apply(lambda text: remove_punctuations(text))

# Remove links
train.text=train.text.apply(lambda text: remove_links(text))

# Remove tags
train.text=train.text.apply(lambda text: remove_tags(text))

# Remove multiple spaces
train.text=train.text.apply(lambda text: remove_multiplespaces(text))

# Remove special characters
train.text=train.text.apply(lambda text: remove_specialchars(text))

# Remove stopwords
train.text=train.text.apply(lambda text: remove_stopwords(text))

# Apply Stemming
train.text=train.text.apply(lambda text: word_stemming(text))

### Cleaning and normalizing validation dataset

In [ ]:
# Covert text to lowercase
test.text=test.text.apply(lambda text: lower_text(text))

# Remove newlines
test.text=test.text.apply(lambda text: remove_newline(text))

# Remove punctuations
test.text=test.text.apply(lambda text: remove_punctuations(text))

# Remove links
test.text=test.text.apply(lambda text: remove_links(text))

# Remove tags
test.text=test.text.apply(lambda text: remove_tags(text))

# Remove multiple spaces
test.text=test.text.apply(lambda text: remove_multiplespaces(text))

# Remove special characters
test.text=test.text.apply(lambda text: remove_specialchars(text))

# Remove stopwords
test.text=test.text.apply(lambda text: remove_stopwords(text))

# Apply Stemming
test.text=test.text.apply(lambda text: word_stemming(text))



## Text visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(20, 12))

wc_disaster = WordCloud(
        width=800, height=600,
        background_color='white',
        stopwords=STOPWORDS
    ).generate(' '.join(train[train.target==1]['text']))

wc_nondisaster = WordCloud(
        width=800, height=600,
        background_color='white',
        stopwords=STOPWORDS
    ).generate(' '.join(train[train.target==0]['text']))

ax1.imshow(wc_disaster)
ax1.set_title("Word cloud of disaster tweets", fontsize=20)
ax1.axis("off")

ax2.imshow(wc_nondisaster)
ax2.set_title("Word cloud of non disaster tweets", fontsize=20)
ax2.axis("off")

fig.show()

## Text vectorization

To process the text data first we need to convert text data into numerical representation for systems to learn further from data. Vectorization is the process of converting a word ito a vector of numbers that contains the information in the word.

We will try:
* Bag-of-words vectorization
* TF-IDF vectorization

In [ ]:
# Bag-of-words
count_vectorizer = CountVectorizer()
train_vectors_bow = count_vectorizer.fit_transform(train["text"])
test_vectors_bow = count_vectorizer.transform(test["text"])

# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
train_vectors_tf = tfidf_vectorizer.fit_transform(train["text"])
test_vectors_tf = tfidf_vectorizer.transform(test["text"])



## Training model

We will use logistic regression to train this model
.

In [ ]:
# Bag-of-words
clf = LogisticRegression()

print("Bag-of-words:\n")
scores = cross_val_score(clf, train_vectors_bow, train["target"], cv=5, scoring="f1")
for k, score in zip(range(len(scores)),scores):
    print("F1 Score for fold %d is %.2f " % (k+1,score))
    
clf.fit(train_vectors_bow, train["target"])

# TF-IDF
clf = LogisticRegression()

print("\nTF-IDF:\n")
scores = cross_val_score(clf, train_vectors_tf, train["target"], cv=5, scoring="f1")
for k, score in zip(range(len(scores)),scores):
    print("F1 Score for fold %d is %.2f " % (k+1,score))
    
clf.fit(train_vectors_tf, train["target"])

### Making predictions using model


In [ ]:
# Bag-of-words
test["pred"] = clf.predict(test_vectors_bow)

print("We got %.1f%% accuracy on our test dataset" % (float(accuracy_score(test.target, test.pred))*100))




In [ ]:
# Bag-of-words
tn, fp, fn, tp = confusion_matrix(test.target, test.pred).ravel()
tot = confusion_matrix(test.target, test.pred).sum()

print("True Negative Rate: %.1f%%" % ((tn/tot)*100))
print("False Positive Rate: %.1f%%" % ((fp/tot)*100))
print("False Negative Rate: %.1f%%" % ((fn/tot)*100))
print("True Positive Rate: %.1f%%" % ((tp/tot)*100))



In [ ]:
# TF-IDF
test["pred"] = clf.predict(test_vectors_tf)

print("We got %.1f%% accuracy on our test dataset" % (float(accuracy_score(test.target, test.pred))*100))


In [ ]:
# TF_IDF
tn, fp, fn, tp = confusion_matrix(test.target, test.pred).ravel()
tot = confusion_matrix(test.target, test.pred).sum()

print("True Negative Rate: %.1f%%" % ((tn/tot)*100))
print("False Positive Rate: %.1f%%" % ((fp/tot)*100))
print("False Negative Rate: %.1f%%" % ((fn/tot)*100))
print("True Positive Rate: %.1f%%" % ((tp/tot)*100))



### Making predictions from competition test data

In [ ]:
# Import data
comp_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

# Check for missing data
comp_test.isna().sum()

In [ ]:
comp_test

In [ ]:
# Isolating text column
test=comp_test[['text','id']]

In [ ]:
## Cleaning and normalizing data

# Covert text to lowercase
test.text=test.text.apply(lambda text: lower_text(text))

# Remove newlines
test.text=test.text.apply(lambda text: remove_newline(text))

# Remove punctuations
test.text=test.text.apply(lambda text: remove_punctuations(text))

# Remove links
test.text=test.text.apply(lambda text: remove_links(text))

# Remove tags
test.text=test.text.apply(lambda text: remove_tags(text))

# Remove multiple spaces
test.text=test.text.apply(lambda text: remove_multiplespaces(text))

# Remove special characters
test.text=test.text.apply(lambda text: remove_specialchars(text))

# Remove stopwords
test.text=test.text.apply(lambda text: remove_stopwords(text))

# Apply Stemming
test.text=test.text.apply(lambda text: word_stemming(text))


In [ ]:
## Predictions


test_vectors_tf = tfidf_vectorizer.transform(test["text"])

test["pred"] = clf.predict(test_vectors_tf)


## Exporting predictions

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

sample_submission["target"] = test["pred"]

sample_submission.to_csv("submission.csv", index=False)

sample_submission[:25]





In [ ]:
sample_submission